## Diferenças entre os arquivos baixados 
- **Streaming_History_Audio[...]**: Histórico de áudio, sendo eles músicas, podcasts e audiobooks.
> Contém músicas (track_name preenchido), pode conter podcasts (episode_name preenchido) e tem ms_played, shuffle, skipped, etc.
> O que vem após o "Audio" é os anos que os dados abrangem e a partição deles (Spotify separa grandes volumes de dados em partes)
- **Streaming_History_Video[...]**: Histórico de conteúdos em formato de vídeos, como podcasts com vídeo, clipes e conteúdos visuais do Spotify.
> Muitas vezes não tem música (track_name=null) e ms_played muito maior

- Não vamos utilizar os dados de vídeo, apenas de áudio nessa análise

In [9]:
# Importar bibliotecas
import pandas as pd
import json
import glob


In [10]:
# Puxando arquivos de áudio 

audio_files = glob.glob("Streaming_History_Audio*.json")

dfs = []

for file in audio_files:
    with open(file, "r", encoding="utf-8") as f:
        data = json.load(f)
        df_temp = pd.DataFrame(data)
        df_temp["source_file"] = file  # opcional
        dfs.append(df_temp)


# Mesclando áudios 
df_audio = pd.concat(dfs, ignore_index=True)


# Mostrando o dataframe final de áudio
df_audio.head(5)


,ts,platform,ms_played,conn_country,ip_addr,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,episode_name,...,audiobook_chapter_uri,audiobook_chapter_title,reason_start,reason_end,shuffle,skipped,offline,offline_timestamp,incognito_mode,source_file
0,2023-08-28T10:02:43Z,ios,53614,BR,189.96.231.37,Shinunoga E-Wa,Fujii Kaze,HELP EVER HURT NEVER,spotify:track:0o9zmvc5f3EFApU52PPIyW,None,...,None,None,clickrow,endplay,True,True,False,1693216908,False,Streaming_History_Audio_2023-2025_0.json
1,2023-08-28T10:03:06Z,ios,22755,BR,189.96.231.37,Matsuri,Fujii Kaze,LOVE ALL SERVE ALL,spotify:track:7AMGgAPFczs3wJgMqu6Eqi,None,...,None,None,clickrow,endplay,True,True,False,1693216963,False,Streaming_History_Audio_2023-2025_0.json
2,2023-08-28T10:03:49Z,ios,35805,BR,189.96.231.37,OTONABLUE,ATARASHII GAKKO!,ICHIJIKIKOKU,spotify:track:3h1XlHgx0m1dO6nNSO1kSV,None,...,None,None,clickrow,fwdbtn,True,True,False,1693216986,False,Streaming_History_Audio_2023-2025_0.json
3,2023-08-28T10:03:52Z,ios,2182,BR,189.96.231.37,絆ノ奇跡,MAN WITH A MISSION,絆ノ奇跡,spotify:track:2VBLFxCUyFp5BfmsZpxcis,None,...,None,None,fwdbtn,endplay,True,True,False,1693217030,False,Streaming_History_Audio_2023-2025_0.json
4,2023-08-28T10:04:30Z,ios,26656,BR,189.96.231.37,OTONABLUE,ATARASHII GAKKO!,ICHIJIKIKOKU,spotify:track:3h1XlHgx0m1dO6nNSO1kSV,None,...,None,None,clickrow,endplay,True,True,False,1693217032,False,Streaming_History_Audio_2023-2025_0.json


In [12]:
# Limpeza do dataset unificado 

# Converter timestamp
df_audio["ts"] = pd.to_datetime(df_audio["ts"])

# Criar colunas de tempo 
df_audio["year"] = df_audio["ts"].dt.year
df_audio["month"] = df_audio["ts"].dt.month
df_audio["date"] = df_audio["ts"].dt.date

# Converter tempo para minutos 
df_audio["minutes_played"] = df_audio["ms_played"] / 60000

# Diferenciar música de podcast 
df_audio["content_type"] = df_audio.apply(
    lambda x: "podcast" if pd.notna(x["episode_name"]) else "music",
    axis=1
)
 
# Ver informações gerais do dataset unificado limpo
df_audio.info()
df_audio.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22383 entries, 0 to 22382
Data columns (total 29 columns):
 #   Column                             Non-Null Count  Dtype              
---  ------                             --------------  -----              
 0   ts                                 22383 non-null  datetime64[ns, UTC]
 1   platform                           22383 non-null  object             
 2   ms_played                          22383 non-null  int64              
 3   conn_country                       22383 non-null  object             
 4   ip_addr                            22383 non-null  object             
 5   master_metadata_track_name         22148 non-null  object             
 6   master_metadata_album_artist_name  22148 non-null  object             
 7   master_metadata_album_album_name   22148 non-null  object             
 8   spotify_track_uri                  22148 non-null  object             
 9   episode_name                       235 non-null   

,ms_played,offline_timestamp,year,month,minutes_played
count,2.238300e+04,2.238300e+04,22383.000000,22383.000000,22383.000000
mean,1.450352e+05,1.732979e+09,2024.373945,7.006165,2.417253
std,1.408096e+05,2.164395e+07,0.694068,3.408985,2.346827
min,0.000000e+00,1.693217e+09,2023.000000,1.000000,0.000000
25%,2.620200e+04,1.714036e+09,2024.000000,4.000000,0.436700
50%,1.743860e+05,1.731708e+09,2024.000000,8.000000,2.906433
75%,2.102400e+05,1.754507e+09,2025.000000,10.000000,3.504000
max,8.323605e+06,1.769714e+09,2026.000000,12.000000,138.726750


In [13]:
# Renomear as colunas 
column_rename_map = {
    "ts": "played_at",
    "platform": "platform_used",
    "ms_played": "milliseconds_played",
    "conn_country": "connection_country",
    "ip_addr": "ip_address",

    "master_metadata_track_name": "track_name",
    "master_metadata_album_artist_name": "artist_name",
    "master_metadata_album_album_name": "album_name",
    "spotify_track_uri": "track_uri",

    "episode_name": "podcast_episode_name",
    "episode_show_name": "podcast_show_name",
    "spotify_episode_uri": "podcast_episode_uri",

    "audiobook_title": "audiobook_title",
    "audiobook_uri": "audiobook_uri",
    "audiobook_chapter_uri": "audiobook_chapter_uri",
    "audiobook_chapter_title": "audiobook_chapter_title",

    "reason_start": "play_start_reason",
    "reason_end": "play_end_reason",

    "shuffle": "shuffle_enabled",
    "skipped": "skipped_track",
    "offline": "offline_playback",
    "offline_timestamp": "offline_timestamp",

    "incognito_mode": "incognito_mode",

    "source_file": "source_file",
    "year": "year",
    "month": "month",
    "date": "date",

    "minutes_played": "minutes_played",
    "content_type": "content_type"
}


df_audio = df_audio.rename(columns=column_rename_map)

df_audio.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22383 entries, 0 to 22382
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype              
---  ------                   --------------  -----              
 0   played_at                22383 non-null  datetime64[ns, UTC]
 1   platform_used            22383 non-null  object             
 2   milliseconds_played      22383 non-null  int64              
 3   connection_country       22383 non-null  object             
 4   ip_address               22383 non-null  object             
 5   track_name               22148 non-null  object             
 6   artist_name              22148 non-null  object             
 7   album_name               22148 non-null  object             
 8   track_uri                22148 non-null  object             
 9   podcast_episode_name     235 non-null    object             
 10  podcast_show_name        235 non-null    object             
 11  podcast_episode_uri      235

## Descrição das variáveis do dataset de Streaming do Spotify
### Informações temporais
- played_at: data e hora (UTC) em que a reprodução foi finalizada
- year: ano da reprodução
- month: mês da reprodução
- date: data da reprodução (sem horário)

### Consumo de conteúdo
- milliseconds_played: tempo total reproduzido em milissegundos
- minutes_played: tempo total reproduzido em minutos
- skipped_track: indica se o conteúdo foi pulado pelo usuário

### Contexto de uso
- platform_used: plataforma utilizada (iOS, Android, Desktop, etc.)
- connection_country: país de conexão no momento da reprodução
- ip_address: endereço IP utilizado
- offline_playback: indica se o conteúdo foi reproduzido offline
- offline_timestamp: timestamp relacionado ao uso offline
- incognito_mode: indica se a sessão estava em modo anônimo

### Informações de música
- track_name: nome da música
- artist_name: nome do artista
- album_name: nome do álbum
- track_uri: identificador único da música no Spotify

### Informações de podcast
- podcast_episode_name: nome do episódio do podcast
- podcast_show_name: nome do programa de podcast
- podcast_episode_uri: identificador único do episódio

### Informações de audiobook
(não utilizadas neste dataset)
- audiobook_title
- audiobook_uri
- audiobook_chapter_uri
- audiobook_chapter_title

### Comportamento de reprodução
- play_start_reason: motivo do início da reprodução (ex: clique do usuário, autoplay)
- play_end_reason: motivo do fim da reprodução (ex: término natural, pular faixa)
- shuffle_enabled: indica se o modo aleatório estava ativado

### Metadados auxiliares
- content_type: tipo de conteúdo (music ou podcast)
- source_file: arquivo de origem do registro (controle de ingestão)